In [1]:
!wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
!tar xvfz Python-3.7.0.tgz
!Python-3.7.0/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_calltips.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_codecontext.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_colorizer.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config_key.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_configdialog.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_debugger.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_delegator.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editmenu.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editor.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_grep.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help_a

In [2]:
!pip install transformers == 3.5
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

ERROR: Invalid requirement: '=='
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

In [3]:
from google.colab import files
dataset = files.upload()

Saving dataset_single.txt to dataset_single.txt


In [4]:
train = pd.read_csv("dataset_single.txt", names=['src', 'tar'], sep="\t")
train = train.reset_index()
train


,index,src,tar
0,1,본관,B_LOC
1,1,본관,B_LOC
2,2,에,O
3,3,가고,O
4,4,싶어,O
...,...,...,...
30501,5,교육,B_LOC
30502,6,관,B_LOC
30503,7,으로,O
30504,8,가고,O


In [5]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]

In [6]:
print(data[:20])

[[1, '본관', 'B_LOC'], [1, '본관', 'B_LOC'], [2, '에', 'O'], [3, '가고', 'O'], [4, '싶어', 'O'], [1, '본관', 'B_LOC'], [2, '에', 'O'], [3, '갈거야', 'O'], [1, '본관', 'B_LOC'], [2, '에', 'O'], [3, '갈', 'O'], [4, '예정', 'O'], [5, '이야', 'O'], [1, '본관', 'B_LOC'], [2, '에', 'O'], [3, '가고', 'O'], [4, '싶어', 'O'], [1, '본관', 'B_LOC'], [2, '갈래', 'O'], [1, '본관', 'B_LOC']]


In [7]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [8]:
print(index_to_ner)

{0: 'B_LOC', 1: 'O', 2: '[PAD]'}


In [9]:
print(label_dict)

{'B_LOC': 0, 'O': 1, '[PAD]': 2}


In [10]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:

  if i != 1:
    temp_tup.append([j,label_dict[k]])
  if i == 1:
    if len(temp_tup) != 0:
      tups.append(temp_tup)
      temp_tup = []
      temp_tup.append([j,label_dict[k]])

tups.pop(0)

[['본관', 'B_LOC']]

In [11]:
print(tups[0], tups[100])

[['본관', 0]] [['풍', 0], ['환경', 0], ['실험', 0], ['관', 0], ['에', 1], ['가고', 1], ['싶어', 1]]


In [12]:
sentences = []
targets = []
for tup in tups:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['O'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['O'])
  sentences.append(sentence)
  targets.append(target)

In [13]:
sentences[20]

['[CLS]', '학생', '복지관', '에', '가고', '싶어', '[SEP]']

In [14]:
targets[20]

[1, 0, 0, 1, 1, 1, 1]

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer.json


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

In [16]:
tokenizer.tokenize("대한민국 만세.")

['대한민국', '만', '##세', '.']

In [17]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [18]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [19]:
print(tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

[(['[CLS]', '본', '##관', '[SEP]'], [1, 0, 0, 1]), (['[CLS]', '본', '##관', '에', '가', '##고', '싶', '##어', '[SEP]'], [1, 0, 0, 1, 1, 1, 1, 1, 1])]


In [20]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [21]:
tokenized_texts[150]

['[CLS]', '디', '##자인', '교', '##육', '관', '에', '갈', '##래', '[SEP]']

In [22]:
labels[150]

[1, 0, 0, 0, 0, 0, 1, 1, 1, 1]

In [23]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))
max_len = 19
bs = 32

19.0


In [24]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")


In [25]:
input_ids[1]

array([  101,  9358, 20595,  9559,  8843, 11664,  9495, 12965,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [26]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [27]:
tags[200]

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [28]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

In [29]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)


In [30]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)


In [31]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(15, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)

  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)

  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [32]:
nr_model = create_model()
nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=10, shuffle=False, batch_size=16)##epoch랑 batch size


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abso

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/pytorch_model.bin
PyTorch checkpoint contains 270,378,749 parameters
Loaded 177,853,440 parameters in the TF 2.0 model.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertFor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 19)]                 0         []                            
 )                                                                                                
                                                                                                  
 input_masks (InputLayer)    [(None, 19)]                 0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1778534   ['input_word_ids[0][0]',      
 )                           ngAndCrossAttentions(last_   40         'input_masks[0][0]']         
                             hidden_state=(None, 19, 76                                       

249/249 [==============================] - 78s 143ms/step - loss: 1.2222 - sparse_categorical_accuracy: 0.4052 - val_loss: 1.0880 - val_sparse_categorical_accuracy: 0.4207
Epoch 2/10
249/249 [==============================] - 30s 121ms/step - loss: 1.0894 - sparse_categorical_accuracy: 0.4200 - val_loss: 1.0840 - val_sparse_categorical_accuracy: 0.4207
Epoch 3/10
249/249 [==============================] - 30s 122ms/step - loss: 1.0845 - sparse_categorical_accuracy: 0.4265 - val_loss: 1.0842 - val_sparse_categorical_accuracy: 0.4207
Epoch 4/10
249/249 [==============================] - 30s 121ms/step - loss: 1.0822 - sparse_categorical_accuracy: 0.4319 - val_loss: 1.0825 - val_sparse_categorical_accuracy: 0.4207
Epoch 5/10
249/249 [==============================] - 30s 121ms/step - loss: 1.0772 - sparse_categorical_accuracy: 0.4385 - val_loss: 1.0854 - val_sparse_categorical_accuracy: 0.4207
Epoch 6/10
249/249 [==============================] - 30s 119ms/step - loss: 1.0770 - sparse_cat

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


In [35]:
y_predicted = nr_model.predict([val_inputs, val_masks])


14/14 [==============================] - 4s 53ms/step


In [36]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

In [37]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

              precision    recall  f1-score   support

       B_LOC       0.00      0.00      0.00      2438
           O       0.42      1.00      0.59      3533

   micro avg       0.42      0.59      0.49      5971
   macro avg       0.21      0.50      0.30      5971
weighted avg       0.25      0.59      0.35      5971



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
def ner_inference(test_sentence):


  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [39]:
ner_inference("나는 학술 정보관 가고 싶어")

1/1 [==============================] - 0s 39ms/step
O	나는
O	학
O	술
O	정
O	보
O	관
O	가
O	고
O	싶
O	어
